# Bibliotecas utilizadas

In [1]:
import pandas as pd
import sqlite3
import requests
import bs4
import time

from IPython.display import IFrame

# Coleta de dados: Web Scraping

## Minha identificação ao usar o navegador para acessar sites com o Google Chrome

In [2]:
# Identificação do "Usuário agente"
agente = 'xxxxxxxxxxxxxxxxxxx'

# Criando chave de identificação
headers = { 'User-Agente': agente }

## Processo de scraping

### **Coletando HTML de site**
<font size = '3'>
Essa atividade será executada pela função 'htmlSite', que fará a requisição do código HTML do site e ajustá-lo.<br>

Obs:<br>

Para que a função funcione, é necessário o Usuário agente (headers) do seu navegador (Chrome), para que <br>
possa ser requerido o conteúdo do site, que virá em formato de byte. A partir disso transformamos o <br>
formato byte usando a codificação correta, tornando-o texto e, em seguida, estruturando-o como objeto <br>
BeautifulSoup, para futuras consultas de tags.    
</font>

In [3]:
# Função para extrair o código HTML do site de interesse
def htmlSite( url: str, decode: str = 'latin1' ) -> str:

    """
    Argumentos:
        url    (str): link do site de interesse;\n
        decode (str): codificação para converter o conteúdo html de bytes para texto
    """

    try:
        # Fazendo requisição de acesso ao html do link
        requisicao = requests.get( url, headers = headers, timeout = 4 )

        # Conteúdo do site ilegível e em formato byte
        resposta_bytes = requisicao.content

        # Conteúdo do site em formato de texto, ainda ilegível
        html_pagina = resposta_bytes.decode( decode )

        # Texto html corrido, separado por tags, perfeitamente legível
        return bs4.BeautifulSoup( html_pagina, 'html.parser' )

    # Caso falhe, apenas passe, não trave a execução do scrip geral
    except: pass

A seguir, um exemplo do que essa função faz.

Vamos entrar no site da câmara de vereadores e consultar uma página de uma proposição que se refere a uma indicação qualquer:

In [4]:
# Url para exemplo
exemplo = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=300&aaProposicao=2007'

# Visualizando página do link
IFrame( exemplo, width = '800', height = '400' )

Agora, **resumo** do conteúdo HTML da página:

In [5]:
# Usando a função criada, com o método prettify para identar o código
html = htmlSite( exemplo ).prettify()

# Mostrando o código html do site (apenas as primeiras linhas)
print( html[:946] )

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <link href="legis.ico" rel="shortcut icon" type="image/x-ico"/>
  <title>
   Câmara Municipal de Indaial _ Indicação nº 300/2007 de 20/03/2007
  </title>
  <meta content="Câmara Municipal de Indaial _ Indicação nº 300/2007 de 20/03/2007" name="description">
   <link href="css/geral3.css" rel="stylesheet"/>
   <link href="https://d11gitgevq44cw.cloudfront.net/libs/font-awesome/5x/css/all.min.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
   <script crossorigin="anonymous" integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8=" src="https://code.jquery.com/jquery-3.3.1.min.js">
   </script>
   


### **Dados a serem extraídos**
<font size = '3'>
Tendo o HTML em mãos, podemos agora extrair seus dados.

No exemplo a seguir, veremos que nossos dados estarão armazenados nas tags 'dt' (colunas) e 'dd' (linha ou dados), as quais<br>
podemos encontrar usando o método '.find_all()'.
</font>


In [6]:
# Url para exemplo
exemplo = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=300&aaProposicao=2007'

# Conteúdo HTML do site
html = htmlSite( exemplo )

# Mostrando todas as tags 'dt' e 'dd' e seus respectivos conteúdos
print( html.find_all( 'dt' ) )
print( html.find_all( 'dd' ) )

[<dt class="col-sm-3">Reunião</dt>, <dt class="col-sm-3">Deliberação</dt>, <dt class="col-sm-3">Situação</dt>, <dt class="col-sm-3">Assunto</dt>, <dt class="col-sm-3">Autor</dt>]
[<dd class="col-sm-9">20/03/2007</dd>, <dd class="col-sm-9">20/03/2007</dd>, <dd class="col-sm-9">Proposição Aprovada</dd>, <dd class="col-sm-9">Abrigo de passageiros</dd>, <dd class="col-sm-9">Vereador <br/><b>Rubens Reinhold Ittner</b>.</dd>]


Para ficar mais visível os valores:

In [7]:
# Mostrando os respectivos conteúdos das tags 'dt' e 'dd'
print( [ i.get_text() for i in html.find_all( 'dt' ) ] )
print( [ i.get_text() for i in html.find_all( 'dd' ) ] )

['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor']
['20/03/2007', '20/03/2007', 'Proposição Aprovada', 'Abrigo de passageiros', 'Vereador Rubens Reinhold Ittner.']


Podemos verificar que é o mesmo conteúdo que conseguimos ver na página formatado como tabela.

### **Extração de dados do HTML**
<font size = '3'>
Essa atividade será executada pela função 'dadosProposicao', que extrairá dados de tags de <br>
'dt' e 'dd' e armazenará em um DataFrame pandas.<br>

A url das proposições segue um padrão, onde 1º temos o tipo de proposição, logo após o número dela e, por fim, <br>
o ano em que a respectiva foi realizada; dessa forma, com o uso de iterações podemos fazer requisições em larga <br>
escala e extrair os dados. <br>

Um ponto de obervação que devemos ter é a possibilidade de a página requisitada não retornar dados, por exemplo: <br>
no ano de 20xx tiveram 1.000 proposições, e na nossa requisição procuramos pela número 1.001... ela irá falhar. <br>
Dessa forma, identifiquei dois padrões: quando a proposição requisitada não existe ou não está disponível e armazenei <br>
na variável 'condicoes'. Após isso, com auxílio de um if, construí um bloco de verificação.<br>

O projeto realizado só focará na extrações de proposições referentes a indicações.
</font>

In [8]:
# Função para, a partir do HTML do site da prefeitura, coletar dados das tags de interesse e armazenar em um DataFrame
def dadosProposicao( ano: int, num_proposicao: int, tipo_proposicao: int = 1 ) -> pd.DataFrame:

    """
    Argumentos:
        ano               (int): ano em que as proposições foram realizadas;\n
        num_proposicao    (int): Define a partir de qual proposição iniciará a extração;\n
        tipo_proposicao   (int): definir se a proposição é uma indicação (1), requerimento (2) ou moção (3).
    """

    # Url do site, para encontrar a preposição
    site = f'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao={ tipo_proposicao }&nrProposicao={ num_proposicao }&aaProposicao={ ano }'

    # HTML do site
    html = htmlSite( site )

    # VERIFICAÇÃO DE PROPOSIÇÃO INEXISTENTE ----------------------------------------------

    condicoes = ( ( html.find_all( 'dt' ) == []               ) or # Verifica se as tags onde guardam os nomes das colunas existem
                  ( 'Proposição não existente' in str( html ) ) )  # Verifica se a substring está contida no HTML

    # COLETANDO DADOS DO SITE ------------------------------------------------------------

    # Se a proposição não existir, retorne um erro no output; se existir, colete os dados
    if  condicoes: raise ValueError()

    else:
        # Criando estrutura do DataFrame esperado
        dfEstrutural = pd.DataFrame( columns = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'] )

        # Coletando apenas tags de interesse
        html_colunas = html.find_all( 'dt' ) # No HTML do site, 'dt' é a tag com o nome das colunas 
        html_linha   = html.find_all( 'dd' ) # No HTML do site, 'dd' é a tag em que o conteúdo da linha está contido

        # Gerando DataFrame com todos os textos contidos nas tags de interesse
        dados = pd.DataFrame( columns = [   i.get_text() for i in html_colunas  ],
                              data    = [ [ i.get_text() for i in html_linha  ] ] ) # É necessário envolver em lista dupla

        # Registrando informações que não estão nas tags consultadas
        dados['Proposição'] = num_proposicao
        dados['Ano'       ] = ano
        dados['Texto'     ] = html.p.get_text()

        # Concantenando para, caso a proposição não tiver dados de uma coluna, a estrutura da tabela não se alterar
        df = pd.concat( [ dfEstrutural, dados ] )

        # Ajustando tipo de dados das colunas
        df['Reunião'    ] = pd.to_datetime( df['Reunião'    ], dayfirst = True, errors = 'coerce' )
        df['Deliberação'] = pd.to_datetime( df['Deliberação'], dayfirst = True, errors = 'coerce' )
        
        df = df.astype( { 'Reunião'        : str, # str porque o sqlite rejeita dados no formato datetime[ns] do pandas
                          'Deliberação'    : str, # str porque o sqlite rejeita dados no formato datetime[ns] do pandas
                          'Situação'       : str,
                          'Assunto'        : str,
                          'Autor'          : str,
                          'Proposição'     : int,
                          'Ano'            : int,
                          'Texto'          : str } )

    return df

Para exemplificar o resultado da função criada, podemos observar uma tabela (DataFrame pandas), com os mesmos<br>
dados que vimos mais acima na página web, como também no conteúdo das tags HTML.

In [9]:
# Não impor limite de caracteres na esposição de tabelas do pandas
pd.options.display.max_colwidth = None

# Usando os mesmos dados dos exemplos anteriores
dadosProposicao( 2007, 300 )

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,2007-03-20,2007-03-20,Proposição Aprovada,Abrigo de passageiros,Vereador Rubens Reinhold Ittner.,300,2007,"Implantação de um abrigo de passageiros na esquina das ruas Ilse Pequena com Mal. Deodoro da Fonseca, bairro Warnow.*com cópia para Setor de Trânsito.\n"


In [10]:
# Estrutura da planilha
dadosProposicao( 2007, 300 ).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Reunião      1 non-null      object
 1   Deliberação  1 non-null      object
 2   Situação     1 non-null      object
 3   Assunto      1 non-null      object
 4   Autor        1 non-null      object
 5   Proposição   1 non-null      int32 
 6   Ano          1 non-null      int32 
 7   Texto        1 non-null      object
dtypes: int32(2), object(6)
memory usage: 184.0+ bytes


### **Extração e armazenamento dos dados**
<font size = '3'>
Tendo acesso ao HTML e construído o código de extração de seus dados, o foco agora é direcionado <br>
para coleta e inserção dos dados no banco de dados de forma continuada entre os anos e suas respectivas<br>
proposições.<br>

Para isso, duas atividades são necessárias:<br>
1. contrução do banco de dados e da tabela onde os dados serão gravados;
2. construção do loop para consulta e iserção dos dados no banco.

</font>

#### **Banco de dados**

Primeiro, precisamos criar ou nos conectar com o banco de dados e seu objeto executor de comandos.

In [11]:
# Criando ou, caso já exista, conectando-se ao banco de dados do projeto
con = sqlite3.connect( 'indicacoesIndaialSC.db' )

# Objeto para executar comandos e recuperar resultados do banco de dados
cursor = con.cursor()

Agora, precisamos verificar se a tabela que será utilizada no projeto já existe no banco de dados.

In [12]:
# Query para verificar se a tabela 'indicacoes' existe
query = """ SELECT name
            FROM   sqlite_master
            WHERE  type = 'table' AND name = 'indicacoes' """

# Código para executar a query e fazer a verificação no banco de dados
verificacao = cursor.execute( query ).fetchone()

# Se a planilha não existir
if verificacao is None:

    # Query para criação de uma tabela
    criacao = """ CREATE TABLE indicacoes( dt_reuniao     DATETIME,
                                           dt_deliberacao DATETIME,
                                           situacao       TEXT,
                                           assunto        TEXT,
                                           autor          TEXT,
                                           proposicao     INTEGER,
                                           ano            INTEGER,
                                           texto          TEXT ) """
    # Criando tabela
    cursor.execute( criacao )

    # Salvando alteração
    con.commit()

    # Registrando
    print( 'Tabela não existia e foi criada.' )

# Se a planilha existir
else:
    
    # Registrando
    print( 'Tabela já existe.' )

Tabela já existe.


#### **Inserções de dados no banco**
<font size = '3'>
Tendo criado o banco, o objetivo agora é, a partir da possibilidade de extração dos dados, armazená-los<br>
dentro dele.

Para isso, o loop escolhido foi o while que armazenará a seguinte estrutura de repetição:
1. a cada nova requisição, deve-se observar qual a última proposição inserida na tabela que pertence ao<br>
ano corrente e definir que a próxima deve ser a requisitada;
2. tendo extraído os dados do html, devem ser imediantamente armazenados no banco de dados.<br>

A única restrição que o while deve atender para não ter sua execução finalizada é a de erros admissíveis:<br>
se não puder extrair os dados 3 vezes seguidas, o while é encerrado.<br>

Nas situações onde, por exemplo, no ano 2000, a preposição imediatamente posterior a 1º é a 3º,<br>
o erro contabilizado após a falha da requisição da 2º preposição é anulado com a execução sem erros da 3º.<br>
Dessa forma, garantindo que a maior parte dos dados sejam devidamente extraídos.

</font>

In [13]:
# Função para extrair dados de proposições (indicações) e inserí-los no banco de dados
def inserirProposicoes( ano: int, erros_admissiveis: int = 3, seg_espera: float = 1):

    """
    Argumentos:
        ano                 (int): ano em que as proposições foram realizadas;\n
        erros_admissiveis   (int): quantas vezes a requisição pode falhar até entender que não há mais dados.\n
        seg_espera        (float): quantidade de tempo de espera até realizar próxima execução.
    """

    # Variáveis do loop
    erros_ocorridos  = 0
    qtd_consultas    = 1 # Quantidade de consultas realizadas

    # Iteração
    while erros_ocorridos < erros_admissiveis:

        # Mostrar qual é a consulta que está sendo realizada
        print( f'{ qtd_consultas } consulta(s) realizada(s).' )

        # INSERIR DADOS DAS PROPOSIÇÕES DO RESPECTIVO ANO NO BANCO DE DADOS ---------------------------------
        try:
            # Verificar qual a última proposição que foi inserida na tabela para o respectivo ano
            query = f"""
                     SELECT MAX( proposicao )
                     FROM indicacoes
                     WHERE ano = { ano }
                     """

            # Extrair o último valor da consulta, usando iloc e reforçando que o valor é inteiro
            ultProposicaoQuery = pd.read_sql_query( query, con ).iloc[ 0, 0 ]

            # Verificação: se não houver proposições para o respectivo ano, adicionar 1 a variável
            if ultProposicaoQuery == None:
                prox_proposicao = 1

            else:
                prox_proposicao = int( ultProposicaoQuery ) + 1 + erros_ocorridos

            # WebScraping da proposição definida
            df = dadosProposicao( ano, prox_proposicao )

            # Inserindo a proposição no banco de dados
            for i, linha in df.iterrows():

                # Inserindo resultados do Dataframe na tabela do banco de dados
                cursor.execute( "INSERT INTO indicacoes VALUES( ?, ?, ?, ?, ?, ?, ?, ? )",
                                ( linha[ 'Reunião'     ],
                                  linha[ 'Deliberação' ],
                                  linha[ 'Situação'    ],
                                  linha[ 'Assunto'     ],
                                  linha[ 'Autor'       ],
                                  linha[ 'Proposição'  ],
                                  linha[ 'Ano'         ],
                                  linha[ 'Texto'       ]  ) )

            # Salvando as inserções
            con.commit()

            # Caso tenha ocorrido 1 erro com a proposição anterior, e a corrente tenha funcionado,
            # reseta a contagem de erros
            erros_ocorridos = 0

        # Caso a função falhe, registrará o erro
        except: erros_ocorridos += 1
        pass

        # Variáveis incrementais: adicionando 1 a cada nova consulta
        qtd_consultas += 1

        # Aguardar a quantidade de tempo definida para continuar o loop
        time.sleep( seg_espera )

Tendo o código para extrair os dados e inserí-los no banco, basta fazer um loop que faça essa execução ano a ano

In [19]:
# Lista de anos que as proposições devem ser coletadas
anos = [ i for i in range( 1996, 2024 ) ]

print( anos )

[1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [ ]:
# Iteração: em cada ano, extrair dados de todas as suas proposições
for ano in anos:
    
    # Registro
    print( 'Inserindo proposições do ano: ', ano )

    # Inserindo proposições
    inserirProposicoes( ano )

    # Registro
    print( 'Finalizado as inserções de proposições do ano: ', ano )

Com isso, conseguimos extrair e armazenar no banco de dados todas as proposições (indicações) de todos os anos, desde o ínicio do registro em 1996.